In [58]:
import folium
import time
import numpy as np
import pandas as pd
from Distance import DistanceMatrix
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [59]:
df_locations = pd.read_csv('./Data/locations.csv')
df_locations

,name,lat,lon
0,San Diego_CA,32.7157,-117.1611
1,Austin_TX,30.2672,-97.7431
2,Denver_CO,39.7392,-104.9903
3,Seattle_WA,47.6062,-122.3321
4,Charleston_SC,32.7765,-79.9311
5,Portland_ME,43.6615,-70.2793
6,Salt Lake City_UT,40.7608,-111.8910
7,Nashville_TN,36.1627,-86.7816
8,Minneapolis_MN,44.9778,-93.2650
9,Savannah_GA,32.0780,-81.0912


In [60]:
dict_locations = df_locations.to_dict(orient='records')
distancematrix = DistanceMatrix(dict_locations)
distancematrix

[[0, 2416, 1742, 2226, 4503, 5479, 1313, 3641, 3203, 4383],
 [2416, 0, 1614, 3703, 2222, 3691, 2247, 1574, 2187, 2074],
 [1742, 1614, 0, 2133, 3078, 3762, 775, 2135, 1461, 2998],
 [2226, 3703, 2133, 0, 5071, 5194, 1464, 4123, 2911, 5022],
 [4503, 2222, 3078, 5071, 0, 1915, 3853, 951, 2310, 173],
 [5479, 3691, 3762, 5194, 1915, 0, 4429, 2121, 2377, 2076],
 [1313, 2247, 775, 1464, 3853, 4429, 0, 2908, 2060, 3774],
 [3641, 1574, 2135, 4123, 951, 2121, 2908, 0, 1458, 901],
 [3203, 2187, 1461, 2911, 2310, 2377, 2060, 1458, 0, 2312],
 [4383, 2074, 2998, 5022, 173, 2076, 3774, 901, 2312, 0]]

In [61]:
#Assume air freight 850 km/hr
average_speed = 850
time_matrix = np.array(distancematrix) / average_speed
time_matrix = np.round(time_matrix).astype(int)
                       

print(time_matrix)

[[0 3 2 3 5 6 2 4 4 5]
 [3 0 2 4 3 4 3 2 3 2]
 [2 2 0 3 4 4 1 3 2 4]
 [3 4 3 0 6 6 2 5 3 6]
 [5 3 4 6 0 2 5 1 3 0]
 [6 4 4 6 2 0 5 2 3 2]
 [2 3 1 2 5 5 0 3 2 4]
 [4 2 3 5 1 2 3 0 2 1]
 [4 3 2 3 3 3 2 2 0 3]
 [5 2 4 6 0 2 4 1 3 0]]


In [62]:
""" num_locations = time_matrix.shape[0]

# Generate random time windows
def generate_time_windows(num_locations):
    time_windows = []
    for i in range(num_locations):
        start = np.random.randint(0, 10)  # Random start time (hours)
        end = np.random.randint(start + 1, 15)  # Random end time (hours) ensuring it's after the start
        time_windows.append((start, end))
    return time_windows

time_windows = generate_time_windows(num_locations)
print(time_windows)

# Print time windows for each location
for i, (start, end) in enumerate(time_windows):
    print(f"Location {i}: Time Window [{start}, {end}]")
 """

' num_locations = time_matrix.shape[0]\n\n# Generate random time windows\ndef generate_time_windows(num_locations):\n    time_windows = []\n    for i in range(num_locations):\n        start = np.random.randint(0, 10)  # Random start time (hours)\n        end = np.random.randint(start + 1, 15)  # Random end time (hours) ensuring it\'s after the start\n        time_windows.append((start, end))\n    return time_windows\n\ntime_windows = generate_time_windows(num_locations)\nprint(time_windows)\n\n# Print time windows for each location\nfor i, (start, end) in enumerate(time_windows):\n    print(f"Location {i}: Time Window [{start}, {end}]")\n '

In [63]:
""" [(2, 13), (0, 5), (1, 9), (9, 11), (2, 11), (5, 10), (0, 8), (7, 12), (3, 11), (8, 14)]
Location 0: Time Window [2, 13]
Location 1: Time Window [0, 5]
Location 2: Time Window [1, 9]
Location 3: Time Window [9, 11]
Location 4: Time Window [2, 11]
Location 5: Time Window [5, 10]
Location 6: Time Window [0, 8]
Location 7: Time Window [7, 12]
Location 8: Time Window [3, 11]
Location 9: Time Window [8, 14] """

' [(2, 13), (0, 5), (1, 9), (9, 11), (2, 11), (5, 10), (0, 8), (7, 12), (3, 11), (8, 14)]\nLocation 0: Time Window [2, 13]\nLocation 1: Time Window [0, 5]\nLocation 2: Time Window [1, 9]\nLocation 3: Time Window [9, 11]\nLocation 4: Time Window [2, 11]\nLocation 5: Time Window [5, 10]\nLocation 6: Time Window [0, 8]\nLocation 7: Time Window [7, 12]\nLocation 8: Time Window [3, 11]\nLocation 9: Time Window [8, 14] '

In [64]:
def create_data_model():
    data = {}
    data['time_matrix'] = time_matrix
    data['time_windows'] = [(0, 5), (2, 13), (1, 9), (9, 11), (2, 11), (5, 10), (0, 8), (7, 12), (3, 11), (8, 14)]
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data

In [65]:
cumul_data = []
routes = []

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}hours\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}hours")

def get_routes(solution, routing, manager):
    global routes
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

def get_cumul_data(solution, routing, dimension):
  """Get cumulative data from a dimension and store it in an array."""
  # Returns an array cumul_data whose i,j entry contains the minimum and
  # maximum of CumulVar for the dimension at the jth node on route :
  # - cumul_data[i][j][0] is the minimum.
  # - cumul_data[i][j][1] is the maximum.
  global cumul_data
  cumul_data = []
  for route_nbr in range(routing.vehicles()):
    route_data = []
    index = routing.Start(route_nbr)
    dim_var = dimension.CumulVar(index)
    route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      dim_var = dimension.CumulVar(index)
      route_data.append([solution.Min(dim_var), solution.Max(dim_var)])
    cumul_data.append(route_data)
  return cumul_data

def get_routes(solution, routing, manager):
  global routes
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

def main():
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        30,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx == data["depot"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # Add time window constraints for each vehicle start node.
    depot_idx = data["depot"]
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(
            data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
        )

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
        get_cumul_data(solution, routing, time_dimension)
        get_routes(solution, routing, manager)

    else:
       print("Not found")


if __name__ == "__main__":
    main()


Objective: 22
Route for vehicle 0:
0 Time(0,0) -> 6 Time(2,2) -> 2 Time(3,3) -> 8 Time(5,5) -> 5 Time(8,8) -> 9 Time(10,10) -> 4 Time(10,10) -> 7 Time(11,11) -> 1 Time(13,13) -> 0 Time(16,16)
Time of the route: 16hours

Route for vehicle 1:
0 Time(0,0) -> 3 Time(9,9) -> 0 Time(12,12)
Time of the route: 12hours

Total time of all routes: 28hours


In [66]:
cumul_data

[[[0, 0],
  [2, 2],
  [3, 3],
  [5, 5],
  [8, 8],
  [10, 10],
  [10, 10],
  [11, 11],
  [13, 13],
  [16, 16]],
 [[0, 0], [9, 9], [12, 12]]]

In [67]:
routes

[[0, 6, 2, 8, 5, 9, 4, 7, 1, 0], [0, 3, 0]]

In [68]:
def mapping(df,routes):
    # Remove unused routes [0,0]
    filtered_routes = [route for route in routes if len(route) != 2]
    # Map coordinates for the remaining routes
    routes_coordinates = {}
    for i, route in enumerate(filtered_routes):
        coordinates = []
        for path in route:
            coordinates.append([df.iloc[path]['lat'], df.iloc[path]['lon']])
        routes_coordinates[i] = coordinates
    
    return routes_coordinates

coordinates = mapping(df_locations,routes)
coordinates

{0: [[np.float64(32.7157), np.float64(-117.1611)],
  [np.float64(40.7608), np.float64(-111.891)],
  [np.float64(39.7392), np.float64(-104.9903)],
  [np.float64(44.9778), np.float64(-93.265)],
  [np.float64(43.6615), np.float64(-70.2793)],
  [np.float64(32.078), np.float64(-81.0912)],
  [np.float64(32.7765), np.float64(-79.9311)],
  [np.float64(36.1627), np.float64(-86.7816)],
  [np.float64(30.2672), np.float64(-97.7431)],
  [np.float64(32.7157), np.float64(-117.1611)]],
 1: [[np.float64(32.7157), np.float64(-117.1611)],
  [np.float64(47.6062), np.float64(-122.3321)],
  [np.float64(32.7157), np.float64(-117.1611)]]}

In [69]:
# Map
map_center = [df_locations['lat'].mean(), df_locations['lon'].mean()]
custom ="cartodb positron"
map = folium.Map(location=map_center, zoom_start=4, tiles=custom)

# Choose location
locations = df_locations

# Name of map
map_name = "./maps/VRPTW.html"
map.save(map_name)

In [70]:
# Add marker
destination = df_locations.drop(index=0)

for _,row in destination.iterrows():
    folium.Circle(
        location=[row['lat'],row['lon']],
        radius=10000,  # Radius in pixels
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['name'],
    ).add_to(map)

""" folium.Marker(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    popup=df_locations.iloc[0]['name'],
    icon=folium.Icon(color='lightred') 
    ).add_to(map) """

folium.Circle(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    radius=10000,  # Radius in pixels
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    tooltip=df_locations.iloc[0]['name']
).add_to(map)

map.save(map_name)

In [71]:
# Add edge
for coordinate in coordinates.values():
    folium.PolyLine(
        locations=coordinate,
        color="#ff6f00",
        weight=1,
        tooltip="TSP"
    ).add_to(map)

map.save(map_name)